In [1]:
import pandas as pd
import numpy as np
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [11]:
pyspark


<module 'pyspark' from 'c:\\Users\\nktun\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\pyspark\\__init__.py'>

In [14]:
spark = SparkSession.Builder().appName('NYC_TAXI_2020').getOrCreate()

In [15]:
df = spark.read.csv('/Users/nktun/Downloads/2020_Yellow_Taxi_Trip_Data.csv', header=True, inferSchema=True)

In [16]:
spark

In [18]:
df.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1|01/01/2020 12:28:...| 01/01/2020 12:33:...|              1|          1.2|         1|                 N|         238|         239|           1|        6.0|  3.0|    0.5|      1.47|         0.0|                  0.3

Paritioning the table based on tpep_pickup_datetime

In [22]:
df.groupBy('VendorID').avg('trip_distance').show()

+--------+------------------+
|VendorID|avg(trip_distance)|
+--------+------------------+
|    null| 26.49469745592712|
|       1|2.5848638779204234|
|       2|2.8279510774394048|
+--------+------------------+



In [33]:
from pyspark.sql.functions import to_date, to_timestamp
from pyspark.sql.types import DateType, TimestampType


In [71]:
df_new = df.withColumn('pickup_date', to_timestamp(df.tpep_pickup_datetime, "MM/dd/yyyy hh:mm:ss a"))


In [52]:
# Seeing the format of the datetime columns
pickup_time_string = df.select('tpep_pickup_datetime')
pickup_time_string.show(10, truncate=False)

In [75]:
# Ensuring all functions are imported for partitioning purpose
from pyspark.sql.functions import *

In [77]:
df_new.write.partitionBy(df_new.pickup_date).parquet('./')

TypeError: Column is not iterable